In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1

class small_basic_block(keras.layers.Layer):

    def __init__(self,out_channels,name=None,**kwargs):
        super().__init__(**kwargs)
        out_div4=int(out_channels/4)
        self.main_layers = [
            keras.layers.Conv2D(out_div4,(1,1),padding='same',activation='relu'),
            keras.layers.Conv2D(out_div4,(1,3),padding='same',activation='relu'),
            keras.layers.Conv2D(out_div4,(3,1),padding='same',activation='relu'),
            keras.layers.Conv2D(out_div4,(1,1),padding='same',activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU()
        ]
    
    def call(self,input):
        x = input
        for layer in self.main_layers:
            x = layer(x)
        return x

class LPRnet(keras.Model):
    def __init__(self,  **kwargs):
        super().__init__(**kwargs)
        self.cnn_layers= [
            keras.layers.Conv2D(64,(3,3),padding='same',name='main_conv1'),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=1),
            small_basic_block(128),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,1)),
            small_basic_block(256),
            small_basic_block(256),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,1)),
            keras.layers.Dropout(0.5),
            keras.layers.Conv2D(256,(1,4),strides=1,padding='same',name='main_conv2'),
            keras.layers.Dropout(0.5),
            keras.layers.Conv2D(NUM_CLASS,(13,1),padding='same',name='main_conv3'),  
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]

    def call(self,inputs,training=False):
        x = inputs
        for layer in self.cnn_layers:
            x = layer(x)
        return x



layer = small_basic_block(256)
print(layer.weights)

[]


In [2]:
model = LPRnet()

In [3]:
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy')
model.build((1,24,94,3))
model.summary()

Model: "lp_rnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_conv1 (Conv2D)          multiple                  1792      
_________________________________________________________________
batch_normalization_1 (Batch multiple                  256       
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
small_basic_block_1 (small_b multiple                  9472      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
small_basic_block_2 (small_b multiple                  3123

In [7]:
sample = np.random.rand(1,24,94,3)
preds = model.predict(sample)


In [8]:
preds.shape

(1, 4, 88, 35)